In [10]:
import pandas, numpy as np
import xml, itertools, re

In [2]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,27,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
unique =  casos.Diagnostico.unique()
unique = [u for u in unique if isinstance(u, str)]
diagnosis.loc[unique, 'dfg'].to_excel('Venezolanas/DiagnosisFiltered.xlsx')

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:961: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


In [208]:
#Check duplicates by birthData
def candidateToRepeated(p1, p2):
    """
    Selects as candidate if they were born in the same name:
    - they have the same 1st surname
    - [TODO] they have similar height (~2cm), as found in the cases
    
    TODO: beware the case of twins!
    """
    #First surname criterio
    if p1[1].Ape1Afil.strip().lower() == p2[1].Ape1Afil.strip().lower():
        return True
    #TODO: Search height and see it's similar
    if False:
        return True
    #Otherwise return False
    return False
    

#Beware
candidates = []
threshold = '2010'
for c in pacientes.groupby('FechaNac'):
    #Arbitrary threshold for separating between mothers and borns
    if len(c[1]) >= 2 and c[0][:4] < threshold:
        for p1, p2 in itertools.combinations(c[1].iterrows(), r = 2):
            if p1[0] != p2[0] and candidateToRepeated(p1, p2):
                candidates.append([p1[0], p2[0]])
pandas.DataFrame.from_records(candidates).to_csv('repeatedCandidates.csv')

In [4]:
import xml.etree.ElementTree as ET


In [5]:
def getEpicrisis(casoID, registrosByCase):
    return registros[(registros.Caso == casoID) & 
                     (registros.Asunto.str.contains('Epicrisis', case = False) == True)]

In [50]:
floatParse = '[0-9]*\.?[0-9]+'
allEqual = lambda s: all([i == j for i, j in  itertools.combinations(s, r = 2)])

def parseGPCA_and_fum(s):
    parsedGPCA = re.findall('G\s*([0-9]+)\s*P\s*([0-9]+)\s*C\s*([0-9]+)\s*A\s*([0-9]+)', registroString)
    parsedFUM = re.findall('FUM: [0-9]+/[0-9]+/[0-9]+', registroString, flags = re.IGNORECASE)
    return {'fum' : len(parsedFUM) > 0,
            'GPCA' : len(parsedGPCA) > 0, 
            'fum_OK' : allEqual(parsedFUM),
            'GPCA_OK' : allEqual(parsedGPCA),
            'fum_Data' : parsedFUM[0] if parsedFUM else [],
            'parsedGPCA' : parsedGPCA[0] if parsedGPCA else []}

In [51]:
registroString = getEpicrisis('AD129559', registros).RegistroXML.iloc[0]

In [53]:
import tqdm
casos_wo_epi = 0
casos_several_epi = 0
results ={}
for i,c in tqdm.tqdm_notebook(enumerate(registros.groupby('Caso')), total = len(casos)):
    if i % 100 == 0 and (i < 1000 or i % 1000 == 0):
        print(i)
    
    epicrisis= getEpicrisis(c[0], c[1])
    if len(epicrisis) == 0:    
        casos_wo_epi += 1
    elif len(epicrisis) >= 2:    
        casos_several_epi += 1
    else:
        stringRegistro = epicrisis.RegistroXML.iloc[0]
        results[c[0]] = parseGPCA_and_fum(stringRegistro)

0

100
200
300
400
500
600
700
800
900
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000



In [43]:
print('Missing epi: %d' % casos_wo_epi)
print('Several epi: %d' % casos_several_epi)

Missing epi: 9693
Several epi: 437


In [64]:
pd = pandas.DataFrame.from_dict(results, orient = 'index')
np.sum(pd.fum_OK), np.sum(pd.fum), np.sum(pd.GPCA_OK), np.sum(pd.GPCA), len(pd) 

(3940, 3940, 3940, 3940, 3940)

In [19]:
allEqual = lambda s: all([i == j for i, j in  itertools.combinations(s, r = 2)])
allEqual(parsedGPCA)

True

In [279]:
r = xml.dom.minidom.parseString(registroString)
print(r.toprettyxml())

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD129559" CentroA="01" CodigoDiagnosticoRelacionado1="R104" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="Z348" IdPaciente="118111" Modulo="01" Padre="AD129559" PlanAdm="100" Prestador="9068204" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  7-&amp;amp;gt; G2 P1 C0 A0
MENARQUIA: 13 A&amp;#209;OS 
VIDA MARITAL. 15 A&amp;#209;OS
VIDA OBSTETRICA: 17 A&amp;#209;OS
FUM: 19/02/15  HOY 38.6 SEMANAS
CICLOS:  REGULARES 6X30
PLANIFICACION: NO
N&amp;#176; COMPA&amp;#209;EROS SEXUALES: 2
CONTROL PRENATA: 2
TIPO DE SANGRE:  O-( PACIENTE REFIERE HEMOCLASIFICOACION DE ELLA, DEL ESPOSO A+ Y DESCONOCE LA DEL HIJO ANTERIOR)

ECOGRAFIA
19/08/15 EMBARAZO DE 26.3 SEMANAS PARA HOY 39.3 SEMANAS
17/09/15 EMBARAZO DE 31 SEMANAS  PARA HOY 39.6 SEMANAS

In [ ]:
def getDataFromPatient(p):
    """
    Get the patient data
    """
    r = {}
    r['id'] = r.Identificacion
    r['marital'] = p.EstadoCivil
    r['etnia'] = p.Etnia #Map from id to string
    r['estudios'] = p.Escolaridad #Map from id to string
    r['nacimiento'] = p.FechaNac #Map from id to string

In [223]:
rString = registros.RegistroXML[465351]
register = ET.fromstring(rString)

In [236]:
r = xml.dom.minidom.parseString(rString)
print(r.toprettyxml())

<?xml version="1.0" ?>
<C Asunto="Evoluci&amp;#243;n" Caso="AD129559" CentroA="01" CodigoDiagnosticoRelacionado1="Z370" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O839" IdPaciente="118111" Modulo="01" Padre="464998" PlanAdm="100" Prestador="9068204" Raiz="94" Registro="106">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="&amp;lt;P&amp;gt;PACIENTE DE 20 A&amp;#209;OS CON IDX DE:&amp;lt;BR&amp;gt;1. PARTO VAGINAL ASISTIDO (18/11/15) 8:22 PM&amp;lt;BR&amp;gt;2. RNUV&amp;lt;BR&amp;gt;3. PUERPERIO INMEDIATO&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;FC: 78 LPM FR: 17 RPM TA: 110/70 T: 36&amp;lt;BR&amp;gt;&amp;lt;/P&amp;gt;
&amp;lt;P&amp;gt;PACIENTE CONSCIENTE, ALERTA Y ORIENTADA, ENREGULAR ESTADO GENERAL, PALIDEZ GENERALIZADA, PUPILAS ISOC&amp;#211;RICAS NORMOREACTIVAS A LA LUZ, MUCOSAS SECAS Y P&amp;#193;LIDAS, CUELLO SIM&amp;#201;TRICO M&amp;#211;VIL, SIN ADENOPATIAS NI INGURGITACI&amp;#211;N YUGULAR. T&amp;#211;RAX SIMETRICO, EXPANSIB

In [231]:
print(c.attrib)
for c in register.getchildren():
    print(c.attrib)

{'NombreCampo': 'NumCertificadoDefuncion', 'ValorCampo': '', 'NombreTabla': 'CamposTexto'}
{'NombreCampo': 'DescripcionNota', 'ValorCampo': '&lt;P&gt;PACIENTE DE 20 A&#209;OS CON IDX DE:&lt;BR&gt;1. PARTO VAGINAL ASISTIDO (18/11/15) 8:22 PM&lt;BR&gt;2. RNUV&lt;BR&gt;3. PUERPERIO INMEDIATO&lt;BR&gt;&lt;BR&gt;FC: 78 LPM FR: 17 RPM TA: 110/70 T: 36&lt;BR&gt;&lt;/P&gt;\r\n&lt;P&gt;PACIENTE CONSCIENTE, ALERTA Y ORIENTADA, ENREGULAR ESTADO GENERAL, PALIDEZ GENERALIZADA, PUPILAS ISOC&#211;RICAS NORMOREACTIVAS A LA LUZ, MUCOSAS SECAS Y P&#193;LIDAS, CUELLO SIM&#201;TRICO M&#211;VIL, SIN ADENOPATIAS NI INGURGITACI&#211;N YUGULAR. T&#211;RAX SIMETRICO, EXPANSIBLE, MURMULLO VESICULAR CONSERVADO EN AMBOS CAMPOS PULMONARES, CLAROS SIN AGREGADOS, RUIDOS CARDIACOS RITMICOS, BIEN TIMBRADOS, SIN SOPLOS, ABDOMEN ,PERISTALSIS PRESENTE, BLANDO DEPRESIBLE, CON LEVE DOLOR A LA PALPACION, BOLO DE SEGURIDAD DE PINARDNORMOTONICO,SIN SIGNOS DE IRRITACI&#211;N PERITONEAL, GENITALES EXTERNOS NORMOCONFIGURADOS CON

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  


In [ ]:
import xml.etree.ElementTree as ET

def getDataFromRegister(c):
    """
    Get some basic information from the mother:
    - Height
    - Weight
    - G A P C
    
    And from the intervention:
    -Type
    """
    

In [37]:
with pandas.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(pacientes.loc[182117])

Identificacion                                                     VEN16598275
Carnet                                                                     NaN
Nom1Afil                                                               NATHALY
Nom2Afil                                                                   NaN
Ape1Afil                                                              MARTINEZ
Ape2Afil                                                                   NaN
Profesion                                                                 9996
FechaNac                                                   1983-10-26 00:00:00
Sexo                                                                         F
DirAfil                                       SAN FERNANDO SEC MEDELLIN CRA 80
TelRes                                                              3226538786
TelOfic                                                            3.13794e+09
EstadoCivil                                         

In [156]:
casos.loc[casos.Diagnostico == 'O064']

,FechaHora,Reingreso,Paciente,Acompañante,DireccionAcompanante,TelefonoAcompanante,Particular,Diagnostico,Prestador,Auxiliar1,...,RequisitosConfirmados,Embarazada,IdCita,IdEstancia,CantidadEvoluciones,FechaPrimeraEvolucion,FechaUltimaEvolucion,UFUltimaEvolucion,PrestadorUltimaEvolucion,UsuarioCierre
Caso,,,,,,,,,,,,,,,,,,,,,
AD273367,2018-01-30 19:46:00.000,False,184670,NaN,NaN,NaN,Entidad,O064,7.357252e+07,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,YMARTINEZI
AD337342,2019-05-05 08:57:00.000,False,207278,NaN,NaN,NaN,Entidad,O064,7.883096e+06,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,amarrugo
AD338358,2019-05-13 08:34:00.000,False,207590,NaN,NaN,NaN,Entidad,O064,9.154276e+06,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,NSALASF
AD350305,2019-08-13 18:08:00.000,False,212633,NaN,NaN,NaN,Entidad,O064,9.154276e+06,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,CJARAMILLOM
AD352149,2019-08-27 18:10:00.000,False,213349,MARIA,NaN,NaN,Entidad,O064,7.321508e+07,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,MPUELLOB
AD354041,2019-09-09 21:23:00.000,False,214114,NaN,NaN,NaN,Entidad,O064,1.047423e+09,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,JAHUMADA
AD361332,2019-11-09 11:55:00.000,False,216713,NaN,NaN,NaN,Entidad,O064,9.154276e+06,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,JAHUMADA
AD366988,2020-01-08 15:01:00.000,False,219041,NaN,NaN,NaN,Entidad,O064,9.154276e+06,NaN,...,0,False,0,NaN,0,NaN,NaN,NaN,NaN,EGOMEZL


In [157]:
v = []
for i in registros.loc[registros.Caso == 'AD273367', 'RegistroXML']:
    v.append(i)

In [158]:
dom = xml.dom.minidom.parseString(v[-2]) # or xml.dom.minidom.parseString(xml_string)
pretty_xml_as_string = dom.toprettyxml()
print(pretty_xml_as_string)


<?xml version="1.0" ?>
<C Asunto="ALTA MEDICA" Caso="AD273367" CentroA="01" CodigoDiagnosticoRelacionado1="" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="" IdPaciente="184670" Modulo="03" Padre="993433" PlanAdm="100" Prestador="73572518" Raiz="07" Registro="14">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="&amp;lt;P&amp;gt;PACIENTE DE 23 A&amp;#209;OS CON DIAGNOSTICO DE&amp;lt;BR&amp;gt;-ABORTO RETENIDO&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;SUBJETIVO: PACIENTE REFIERE SENTIRSE BIEN, NIEGA DOLOR ABDOMINAL, NIEGA NAUSEAS, VOMITOS,MAREOSUOTRA SINTOMATOLOGIA. &amp;lt;/P&amp;gt;
&amp;lt;P&amp;gt;OBJETIVO FC:80 LPM FR:17 RPM TA:100/70&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;PACIENTE CONSCIENTE, ALERTA, ORIENTADA. NORMOCEFALO, ESCLERAS ANICTERICAS, PUPILAS ISOCORICAS, NORMORREACTIVAS A LA LUZ, MUCOSA ORAL HUMEDA SIN LESIONES. CUELLO SIMETRICO, MOVIL, SIN ADENOPATIAS PALPABLES. TORAX SIMETRICO, EXPANSIBLE,MURMULLO VESICULA

In [140]:


import xml.dom.minidom

for vv in v[1:]:
    dom = xml.dom.minidom.parseString(vv) # or xml.dom.minidom.parseString(xml_string)
    pretty_xml_as_string = dom.toprettyxml()
    print(pretty_xml_as_string)
    break

<?xml version="1.0" ?>
<C Asunto="EVOLCUION CON RESULTADO DE PARACLINICOS " Caso="AD271811" CentroA="01" CodigoDiagnosticoRelacionado1="" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O033" IdPaciente="184101" Modulo="03" Padre="987884" PlanAdm="100" Prestador="7883096" Raiz="07" Registro="14">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="PACIENTE DE 26 A&amp;#209;OS DE EDAD G2 P1 C1&amp;amp;nbsp; A1&amp;amp;nbsp;CON IDX&amp;lt;BR&amp;gt;- ABORTO INCOMPLETO&amp;amp;nbsp;&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;PACIENTE REFIERE SENTIRSE BIEN&amp;amp;nbsp;&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;TA:120/80&amp;amp;nbsp;&amp;amp;nbsp;&amp;amp;nbsp; FC:76&amp;amp;nbsp;&amp;amp;nbsp; FR:18&amp;amp;nbsp;&amp;amp;nbsp;&amp;amp;nbsp;&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;PACIENTE CONSCIENTE ALERTA ORIENTADA HIDRATA, CON PUPILAS ISOCORICAS FOTO REACTICVAS, MUCOSA ORAL HUMEDA HIDRATADA SIN LESIONES, CUELLO MOVIL SIMETRICO NO DOLOROSO A LA PALPACION,

In [72]:
dom = xml.dom.minidom.parseString(v[0]) # or xml.dom.minidom.parseString(xml_string)
dom.findall('Analisis')

TypeError: 'NoneType' object is not subscriptable

In [73]:
v = dom.getElementsByTagName('row')

In [120]:
i = 57
print(i, v[i].attributes['NombreCampo'].value,v[i].nodeValue)
print (" ".join(t.nodeValue for t in v[i].childNodes if t.nodeType == t.TEXT_NODE))

57 Analisis None



In [136]:
from html.parser import HTMLParser
pars =HTMLParser()
pars.unescape(s)

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


'PACIENTE DE 27 AÑOS CON EMBARAZO DE 8 SEMANAS POR FUM + AMENAZA DE ABORTO VS  ABORTO INCOMPLETO, ACTUALMENTE HEMODINAMICAMENTE ESTABLE, AFEBRIL, CON SIGNOS VITALES DENTRO DE PARAMETROS NORMALES, SIN SIGNOS DE SIRS NI IRRITACION PERITONEAL, TOLERANDO OXIGENO AMBIENTE Y VIA ORAL. SE DECIDE INGRESAR A LA PACIENTE PARA REALIZACION DE PARACLINICOS QUE CONFIRMEN DIAGNOSTICO Y REVALORAR.'